In [1]:
%matplotlib inline
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.utils import Sequence
from keras.utils import load_img
from keras.optimizers import RMSprop


2023-01-18 10:28:33.431827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 10:28:37.984222: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /appl/cudnn/v8.3.2.44-prod-cuda-11.5/lib:/appl/cuda/12.0.0/lib64:/appl/python/3.10.7/lib:/appl/gcc/11.3.0-binutils-2.38/lib64:/appl/gcc/11.3.0-binutils-2.38/lib:/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2023-01-18 10:28:37.984276: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-18 10:28

In [2]:
df = pd.read_csv('black_background_500x500.csv')
train_df = df[df['ImagePath'].str.contains("train")]
test_df = df[df['ImagePath'].str.contains("test")]
valid_df = df[df['ImagePath'].str.contains("valid")]

input_size = 500

In [3]:
#########################################################
################## data generator #######################
#########################################################
class datagenerator(tf.keras.utils.Sequence):
    def __init__(self, 
            batch_size, 
            img_size,
            data_paths_df,
            input_channels,
            output_channels):
         
        self.batch_size = batch_size
        self.img_size = img_size
        self.data_paths_df = data_paths_df
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.data_paths = data_paths_df.values[:,1]
        self.params = data_paths_df.values[:,3:6]
        assert len(self.data_paths) == len(self.params)
        
        self.n = len(self.data_paths)

    def on_epoch_end(self):
        'updates indexes after each epoch'
        self.data_paths_df = self.data_paths_df.sample(frac = 1)
        self.data_paths = self.data_paths_df.values[:,1]
        self.params = self.data_paths_df.values[:,3:6]
    
    def __getitem__(self, index):
        batch_data_paths = self.data_paths[index : index + self.batch_size]
        batch_params_paths = self.params[index : index + self.batch_size]

        return self.__dataloader(self.img_size,
                batch_data_paths, batch_params_paths,
                self.input_channels, self.output_channels)
    
    def __len__(self):
        return self.n // self.batch_size

    #################### data loader ########################
    def __dataloader(self, 
            img_size,
            data_paths,
            batch_params_paths,
            input_channels,
            output_channels):
        x = np.zeros((len(data_paths), img_size[0], img_size[1], input_channels))
        y = batch_params_paths        
        
        for i in range(len(data_paths)):
            data = load_img(path = data_paths[i], grayscale = True)
            data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
            data /= 255
            data.shape = (1,) + data.shape
            x[i] = np.asarray(data)
        return x.astype("float32"), np.array(y).astype("float32")

In [4]:
import keras.backend as K

def abs_loss_function(y_true, y_pred):   
    abs_diff = K.abs(y_true - y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(ones - abs_diff )   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
    return tf.math.reduce_mean(minimum_from_two, axis=-1)

def square_abs_min_loss(y_true, y_pred):   
    abs_diff = K.abs(y_true - y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(tf.ones_like(y_true) - abs_diff )   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
    result = tf.math.square( (minimum_from_two[:, 0] + minimum_from_two[:, 1] + minimum_from_two[:, 2]) / 3 )
    return K.mean(result)

def square_abs_min_individual_loss(y_true, y_pred):   
    abs_diff = K.abs(y_true - y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(tf.ones_like(y_true) - abs_diff )   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
    result = (tf.math.square(minimum_from_two[:, 0]) + 
              tf.math.square(minimum_from_two[:, 1]) + 
              tf.math.square(minimum_from_two[:, 2])) / 3 
    return K.mean(result)

############################# For debugging ####################################
#     print("_________________ 1 __________________")
#     print(abs_diff_reversed.numpy())
#     print("_________________ 2 __________________")
#     print(abs_diff.numpy())

In [6]:
from keras.models import load_model 
from tensorflow.keras.utils import CustomObjectScope

with CustomObjectScope({'abs_loss_function': square_abs_min_loss}):
    model = keras.models.load_model('Models/3db071e0968f11ed81960894ef90a55a_model_adam_001.h5')


In [12]:
def prdict_and_print(nr):
    t = test_df.values[nr][1]
    data = load_img(path = t, grayscale = True)
    data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
    data /= 255
    print("@@@@@@@", data[250][50])
    data.shape = (1,) + data.shape
    X = np.asarray(data)
    print("----------{}----------".format(nr))
    euler = t.split("_")
    print("phi1", float(euler[3]))
    print("PHI",   float(euler[4]))
    print("phi2",  float(euler[5][:-4]))
    yhat = model.predict(X)
    print("predicted values", yhat*90)

    
print("############### PREDICTIONS ###############")
for i in range(10):
    prdict_and_print(i)
print("############### PREDICTIONS ###############")

############### PREDICTIONS ###############
@@@@@@@ [0.3647059]
----------0----------
phi1 54.7
PHI 36.1
phi2 23.5
1/1 [==============================] - 0s 43ms/step
predicted values [[ 37.6812     -1.2368847 114.12841  ]]
@@@@@@@ [0.36862746]
----------1----------
phi1 76.0
PHI 83.7
phi2 2.9
1/1 [==============================] - 0s 34ms/step
predicted values [[ 37.6812     -1.2368847 114.12841  ]]
@@@@@@@ [0.3647059]
----------2----------
phi1 17.8
PHI 63.8
phi2 50.8
1/1 [==============================] - 0s 36ms/step
predicted values [[ 37.6812     -1.2368847 114.12841  ]]
@@@@@@@ [0.4392157]
----------3----------
phi1 17.5
PHI 15.5
phi2 50.4
1/1 [==============================] - 0s 33ms/step
predicted values [[ 37.6812     -1.2368847 114.12841  ]]
@@@@@@@ [0.30980393]
----------4----------
phi1 47.5
PHI 32.6
phi2 29.8
1/1 [==============================] - 0s 34ms/step
predicted values [[ 37.6812     -1.2368847 114.12841  ]]
@@@@@@@ [0.42352942]
----------5----------
phi1 33.3
PH